# Satellite-based monitoring of dry and wet conditions using Standardized Precipitation Index (SPI)  colab version

Surajit Ghosh |Rim | IWMI 

The SPI analysis is following the training conducted in 28 Jan 2020 by [NASA ARSET](https://arset.gsfc.nasa.gov) on **Application of GPM IMERG Reanalysis for Assessing Extreme Dry and Wet Periods**. Link: https://arset.gsfc.nasa.gov/water/webinars/IMERG-2020

Some of the step have been modified and adjusted based on experiencing of several problems during the training, and latest version of [Climate Indices in Python](https://github.com/monocongo/climate_indices) software is used in this tutorial. While NASA ARSET training still used the official release version from [U.S. Drought Portal](https://www.drought.gov/drought/python-climate-indices)


## 1. Objectives
- Learn how to bulk download IMERG data from NASA GES DISC
- Determine how to calculate the Standardized Precipitation Index (SPI) for assessing extreme dry and wet periods

### 1.1. Outline
- Background on the Standardized Precipitation Index (SPI)
    - SPI Interpretation
- Software and Data Preparation
    - Case Study: Zambia
    - Install Supporting Software
    - Data Acquisition
    - Configure the Conda and python environment
    - Pre-process data using NetCDF Operator ([NCO](http://nco.sourceforge.net/nco.html))
- Running SPI code    
    - Pre-compute SPI distribution fitting variables (Optional analysis)
- Interpret the result using Panoply

## 2. Background on the Standardized Precipitation Index
First developed by T.B. McKee et al. (1993) and used by Guttman (1999)
- Used for estimating meteorological conditions based on precipitation alone
- Wet or dry conditions can be monitored on a variety of time scales from sub seasonal to interannual
- Can be compared across regions with markedly difference climates
- Does not consider the intensity of precipitation and its potential impacts on runoff, streamflow, and water availability
- Expressed as the number of standard deviations from the long term mean, for a normally distributed random variable, and fitted probability distribution for the actual precipitation record
- SPI values < 1 indicate a condition of drought, the more negative the value the more severe the drought condition. SPI values > +1 indicate wetter conditions compared to a climatology

### 2.1. SPI Interpretation
https://drought.unl.edu/droughtmonitoring/SPI.aspx

- 1 month: Similar to a map displaying the percent of normal precipitation for a month. Reflects relatively short term conditions. Its application can be related closely with short term soil moisture and crop stress.
- 3 month: Provides a comparison of the precipitation over a specific 3 month period with the precipitation totals from the same 3 month period for all the years included in the historical record. Reflects short and medium term moisture conditions and provides a seasonal estimation of precipitation.
- 6 month: Compares the precipitation for that period with the same 6 month period over the historical record. A 6 month SPI can be very effective in showing the precipitation over distinct seasons and may be associated with anomalous streamflow and reservoir levels.
- 9 month: Provides an indication of precipitation patterns over a medium time scale. SPI values below 1.5 for these time scales are usually a good indication that significant impacts are occurring in agriculture and may be showing up in other sectors as well.
- 12 month: Reflects long term precipitation patterns. Longer SPIs tend toward zero unless a specific trend is taking place. SPIs of these time scales are probably tied to streamflow, reservoir levels, and even groundwater levels at the longer time scales. In some locations of the country, the 12 month SPI is most closely related with the Palmer Index, and the two indices should reflect similar conditions.

 (Source: J. Keyantash)

### 2.2. Reference
- Guttman, N. B., 1999: Accepting the Standardized Precipitation Index: A calculation algorithm. J. Amer. Water Resour. Assoc., 35(2), 311-322. [Link](https://climatedataguide.ucar.edu/climate-data/standardized-precipitation-index-spi)
- Keyantash, John & National Center for Atmospheric Research Staff (Eds). "The Climate Data Guide: Standardized Precipitation Index (SPI)." Retrieved from https://climatedataguide.ucar.edu/climate-data/standardized-precipitation-index-spi
- Lloyd Hughes, B., and M. A. Saunders, 2002: A drought climatology for Europe. Int. J. Climatol., DOI:10.1002/joc.846 [Link](https://rmets.onlinelibrary.wiley.com/doi/epdf/10.1002/joc.846)
- McKee, T.B., N. J. Doesken, and J. Kliest, 1993: The relationship of drought frequency and duration to time scales. In Proceedings of the 8th Conference of Applied Climatology, 17 22 January, Anaheim, CA. American Meterological Society, Boston, MA. 179-18. [Link](https://www.droughtmanagement.info/literature/AMS_Relationship_Drought_Frequency_Duration_Time_Scales_1993.pdf)
- National Drought Mitigation Center (NDMC) at the University of Nebraska Lincoln. [Link](https://drought.unl.edu/droughtmonitoring/SPI.aspx)
- World Meteorological Organization (WMO), 2012: Standardized Precipitation Index User Guide. [Link](https://library.wmo.int/doc_num.php?explnum_id=7768)
- Climate Indices in Python https://climate-indices.readthedocs.io/en/latest/

## 3. Software and Data Preparation
Case study: Zambia

### 3.1. Install Supporting Software
**If you encounter a problem, please look for a online solution.** The installation and configuration described below is performed in the colab terminal.  

1. However, to visualize the results, you need to download and install [Panoply Data Viewer](https://www.giss.nasa.gov/tools/panoply/) from [NASA GISS](https://www.giss.nasa.gov/tools/panoply/download/) on your machine:
    - macOS: https://www.giss.nasa.gov/tools/panoply/download/PanoplyMacOS-4.11.4.dmg
    - Windows: https://www.giss.nasa.gov/tools/panoply/download/PanoplyWin-4.11.4.zip
    - Linux: https://www.giss.nasa.gov/tools/panoply/download/PanoplyJ-4.11.4.zip

In [1]:
# new code
!pip install condacolab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached condacolab-0.1.3-py3-none-any.whl (6.8 kB)


In [2]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


2. Install ```wget``` package. ues ```wget``` to download IMERG data and use wget to prepare conda distribution in colab

In [3]:
!pip install wget 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 3.2. Data Acquisition
1. Download monthly IMERG data from GES DISC: More detailed instructions can be found from EARTHDATA website https://disc.gsfc.nasa.gov/data-access
    - Using a web browser, go to NASA Goddard Earth Sciences (GES) Data and Information Services Center (DISC): https://disc.gsfc.nasa.gov/
    - Type “IMERG” in the search bar and click on the search icon
    - Select IMERG Version 6 Level 3 data at “monthly” temporal resolution and click on the “Subset/Get Data” icon   
    - Leave the default date range since we want the entire time series
    - Under Spatial Subset enter ```114.3, -9, 115.8, -8``` This spatial subset is for **Zambia**
    - Under Variables select only ```precipitation```
    - Leave the default parameters under Grid
    - Under File Format select "netCDF"
    - Click Get Data    
    - Data links windows will popup and you may click **"Download links list"**
    - You will get a txt file with similar filename like this one **subset_GPM_3IMERGM_06_20210117_064353.txt**
    - Upload this txt file into your specified download folder in this case folder **IMERG_mm_hr** 
    - Run the code to download the data to folder **IMERG_mm_hr**


    



In [4]:
from google.colab import drive
drive.mount('/content/drive')  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# remember to change the download txt file name, output dir, your username and password
!wget --user=rimu --password=theworldisfullofcrap1A. -P /content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_day --content-disposition -i  /content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_day/subset_GPM_3IMERGDL_06_20220721_202813.txt

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Saving to: ‘/content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_day/3B-DAY-L.MS.MRG.3IMERG.20211113-S000000-E235959.V06.nc4.SUB.nc4’

3B-DAY-L.MS.MRG.3IM 100%[===================>]  66.59K  --.-KB/s    in 0.1s    

2022-07-22 00:13:27 (585 KB/s) - ‘/content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_day/3B-DAY-L.MS.MRG.3IMERG.20211113-S000000-E235959.V06.nc4.SUB.nc4’ saved [68186/68186]

--2022-07-22 00:13:27--  https://gpm1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FGPM_L3%2FGPM_3IMERGDL.06%2F2021%2F11%2F3B-DAY-L.MS.MRG.3IMERG.20211114-S000000-E235959.V06.nc4&FORMAT=bmM0Lw&BBOX=-18.3%2C21.885%2C-8.104%2C33.75&LABEL=3B-DAY-L.MS.MRG.3IMERG.20211114-S000000-E235959.V06.nc4.SUB.nc4&SHORTNAME=GPM_3IMERGDL&SERVICE=L34RS_GPM&VERSION=1.02&DATASET_VERSION=06&VARIABLES=precipitationCal
Connecting to gpm1.gesdisc.eosdis.nasa.gov (gpm1.gesdisc.eosdis.nasa.gov)|198.118.197.50|:443... conn

### 3.3. Configure the Python environment
This code for calculating SPI is written in Python 3. It is recommended to use either the [Miniconda3](https://docs.conda.io/en/latest/miniconda.html) (minimal Anaconda) or Anaconda3 distribution. The below instructions will be Anaconda specific (although relevant to any Python [virtual environment](https://virtualenv.pypa.io/en/stable/)).

A new Anaconda [environment](https://conda.io/docs/using/envs.html) can be created using the [conda](https://conda.io/docs/) environment management system that comes packaged with Anaconda. Note: you may need to restart the kernel to use some updated packages. The ```conda``` bash should be run in colab terminal which is provided for colab pro user.

##### Conda
```conda``` is not preinstalled in the default configuration in Colab. Check the following blog to know more details about how to fix it.

https://rjai.me/posts/google-colab-conda/

In [5]:
# new code
!wget https://repo.anaconda.com/archive/Anaconda3-2020.02-Linux-x86_64.sh && bash Anaconda3-2020.02-Linux-x86_64.sh -bfp /usr/local

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
anaconda==2020.02=py37_0 -> tk==8.6.8=hbc83047_0
seaborn==0.10.0=py_0 -> python[version='>=3.6'] -> tk[version='>=8.6.10,<8.7.0a0|>=8.6.11,<8.7.0a0|>=8.6.8,<8.7.0a0']
pyopenssl==19.1.0=py37_0 -> python[version='>=3.7,<3.8.0a0'] -> tk[version='>=8.6.10,<8.7.0a0|>=8.6.11,<8.7.0a0|>=8.6.8,<8.7.0a0']
sphinxcontrib-serializinghtml==1.1.3=py_0 -> python[version='>=3.5'] -> tk[version='>=8.6.10,<8.7.0a0|>=8.6.11,<8.7.0a0|>=8.6.8,<8.7.0a0']
pycurl==7.43.0.5=py37h1ba5d50_0 -> python[version='>=3.7,<3.8.0a0'] -> tk[version='>=8.6.10,<8.7.0a0|>=8.6.11,<8.7.0a0|>=8.6.8,<8.7.0a0']
click==7.0=py37_0 -> python[version='>=3.7,<3.8.0a0'] -> tk[version='>=8.6.10,<8.7.0a0|>=8.6.11,<8.7.0a0|>=8.6.8,<8.7.0a0']
cryptography==2.8=py37h1ba5d50_0 -> python[version='>=3.7,<3.8.0a0'] -> tk[version='>=8.6.10,<8.7.0a0|>=8.6.11,<8.7.0a0|>=8.6.8,<8.7.0a0']
dask==2.11.0=py_0 -> python[version='>=3.6'] -> tk[version='>=8.6.10,<8.7.0a0|>=8.6.

Any package that you install with Conda will be installed into the directory /usr/local/lib/python3.7/site-packages so you will need to add this directory to sys.path in order for these packages to be available for import.

In [6]:
# new code
import sys
sys.path.insert(0, "/usr/local/lib/python3.7/site-packages/")

Now that you have installed Conda you need to update Conda and all its dependencies to their most recent versions and update Python to 3.7. The conda update command then updates all of Conda’s dependencies to their most recent versions

In [7]:
# new code
!conda install -yq python=3.7

Solving environment: ...working... done

# All requested packages already installed.



#### NCO

NetCDF Operators (NCO) is a requirement and must be installed for utilization of this package. Instructions for installation on various platforms is available [here](http://nco.sourceforge.net/#Executables//) and other information related to NCO command can be found at the link: http://nco.sourceforge.net/nco.html. If using an Anaconda environment as advised above then it’s as simple as running the following command within the activated conda environment:

In [8]:
!conda install -c conda-forge nco

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - nco


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    c-ares-1.18.1              |       h7f98852_0         113 KB  conda-forge
    cudatoolkit-11.1.1         |      ha002fc5_10        1.20 GB  conda-forge
    curl-7.83.1                |       h7bff187_0          89 KB  conda-forge
    esmf-8.2.0                 | nompi_h61edca3_0        27.4 MB  conda-forge
    expat-2.4.8                |       h27087fc_0         187 KB  conda-forge
  

#### Climate-indices package
Install ```climate-indices``` package. Once the environment has been activated then subsequent Python commands will run in this environment where the package dependencies for this project are present. Now the package can be added to the environment along with all required modules (dependencies) via [pip](https://pip.pypa.io/en/stable/):

In [9]:
!pip install climate-indices

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 71 kB 118 kB/s 
     |████████████████████████████████| 15.7 MB 147 kB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 5.1 MB 38.9 MB/s 
     |████████████████████████████████| 1.1 MB 56.4 MB/s 
     |████████████████████████████████| 3.3 MB 39.7 MB/s 
     |████████████████████████████████| 845 kB 46.1 MB/s 
     |████████████████████████████████| 40 kB 4.8 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
     |████████████████████████████████| 140 kB 58.5 MB/s 
     |████████████████████████████████| 98 kB 7.1 MB/s 
     |████████████████████████████████| 231 kB 49.6 MB/s 
     |████████████████████████████████| 34.5 MB 1.2 kB/s 
     |████████████████████████████████| 11.3 MB 45.9 MB/s 
     |████████████████████████████████

### 3.4. Preprocess data using NCO


1. The original downloaded files from GPM IMERG is in mm/day, while to calculate monthly SPI the data must be in mm/month, we need to do a conversion process using  ```ncrcat``` to concatenate the data of the same month and year.

2. ```ncap2``` (arithmetic operator for NetCDF files) is then used to sum the concatenated data. 

In [10]:
%cd /content/drive/MyDrive/IWMI/IMERG_DailyData

/content/drive/MyDrive/IWMI/IMERG_DailyData


In [ ]:
!ncrcat /content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_day/3B-DAY-L.MS.MRG.3IMERG.202201*.nc4 /content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_concat/3B-DAY-L.MS.MRG.3IMERG.20220101-S000000-E235959.V06.nc4.SUB.nc4

In [ ]:
!ncra -h -O -y ttl  /content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_concat/3B-DAY-L.MS.MRG.3IMERG.20220101-S000000-E235959.V06.nc4.SUB.nc4  /content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_month/3B-DAY-L.MS.MRG.3IMERG.20220101-S000000-E235959.V06.nc4.SUB.nc4

In [308]:
!ncdump -h /content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_month//3B-DAY-L.MS.MRG.3IMERG.20020101-S000000-E235959.V06.nc4.SUB.nc4

netcdf \3B-DAY-L.MS.MRG.3IMERG.20020101-S000000-E235959.V06.nc4.SUB {
dimensions:
	time = UNLIMITED ; // (1 currently)
	bnds = 2 ;
	lon = 119 ;
	lat = 102 ;
variables:
	double time(time) ;
		time:standard_name = "time" ;
		time:bounds = "time_bnds" ;
		time:units = "days since 1970-01-01 00:00:00Z" ;
		time:calendar = "standard" ;
		time:axis = "T" ;
		time:cell_methods = "time: mean" ;
	double time_bnds(time, bnds) ;
		time_bnds:cell_methods = "time: mean" ;
	float lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "Longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
	float lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "Latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
	float precipitationCal(time, lon, lat) ;
		precipitationCal:long_name = "Daily accumulated precipitation (combined microwave-IR) estimate" ;
		precipitationCal:units = "mm" ;
		precipitationCal:_FillValue = -9999.9f ;
		precipitationCal:missing_value = -9999.9f ;


3. Navigate to ```IMERG_mm_month``` folder in terminal. Loops all files in the folder ```IMERG_mm_month``` to make ```time``` the record dimension/variable used for concatenating using ```Ncks``` command

In [309]:
%cd /content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_month

/content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_month


In [310]:
!for fl in *.nc4; do ncks --mk_rec_dmn time $fl -o $fl.TMP; mv $fl.TMP $fl; done

INFO: NCO stub function for nc_inq_filter_avail() reports an inquiry on filter availability of HDF5 filter ID = 307. Stub function employed because libnetcdf.a does not contain nc_inq_filter_avail(). Please rebuild NCO against netCDF library version 4.9.0 (released ~20220601) or later to support the capability to find and call filters besides DEFLATE, Shuffle, and Fletcher32.
Exiting...
ncks: WARNING nc_cdc_lst_bld() reports nco_inq_filter_avail() did not find Bzip2 filter (with HDF5 filter ID = 307) as an HDF5 shared library filter. This is probably fixable because this filter is supported by all default installations of netCDF version 4.9.0 or higher. HINT: If you build netCDF from source, please be sure it was configured with the following options: "--enable-nczarr" and "--with-plugin-dir=${HDF5_PLUGIN_PATH}". The latter is especially important in netCDF 4.9.0. Also, please be sure the library for the missing filter (e.g., libzstd.a, libblosc.a, libbz2.a) is in an automatically sear

4. Concatenates all ```.nc4``` files in ```IMERG_mm_month``` folder into one ```.nc4``` file named ```IMERG_concat.nc4``` using ```ncrcat``` command

In [311]:
!ncrcat -h *.nc4 IMERG_concat.nc4

INFO: NCO stub function for nc_inq_filter_avail() reports an inquiry on filter availability of HDF5 filter ID = 307. Stub function employed because libnetcdf.a does not contain nc_inq_filter_avail(). Please rebuild NCO against netCDF library version 4.9.0 (released ~20220601) or later to support the capability to find and call filters besides DEFLATE, Shuffle, and Fletcher32.
Exiting...
ncrcat: WARNING nc_cdc_lst_bld() reports nco_inq_filter_avail() did not find Bzip2 filter (with HDF5 filter ID = 307) as an HDF5 shared library filter. This is probably fixable because this filter is supported by all default installations of netCDF version 4.9.0 or higher. HINT: If you build netCDF from source, please be sure it was configured with the following options: "--enable-nczarr" and "--with-plugin-dir=${HDF5_PLUGIN_PATH}". The latter is especially important in netCDF 4.9.0. Also, please be sure the library for the missing filter (e.g., libzstd.a, libblosc.a, libbz2.a) is in an automatically se

5. Check the header

In [312]:
!ncdump -h IMERG_concat.nc4

netcdf IMERG_concat {
dimensions:
	lat = 102 ;
	lon = 119 ;
	time = UNLIMITED ; // (266 currently)
	bnds = 2 ;
variables:
	float lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "Latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
	float lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "Longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
	float precipitationCal(time, lon, lat) ;
		precipitationCal:long_name = "Daily accumulated precipitation (combined microwave-IR) estimate" ;
		precipitationCal:units = "mm" ;
		precipitationCal:_FillValue = -9999.9f ;
		precipitationCal:missing_value = -9999.9f ;
		precipitationCal:cell_methods = "time: mean time: sum" ;
	double time(time) ;
		time:standard_name = "time" ;
		time:bounds = "time_bnds" ;
		time:units = "days since 1970-01-01 00:00:00Z" ;
		time:calendar = "standard" ;
		time:axis = "T" ;
		time:cell_methods = "time: mean" ;
	double time_bnds(time, bnds) ;
		time_bnds:cell_methods = "t

6. And the variables for ```precipitation``` is ```time,lon,lat``` but SPI calculation required:
    - ```lat,lon,time``` or
    - ```time,lat,lon```
    

   

7. Let's re-order the variables into ```time,lat,lon``` using ```ncpdq``` command, to be able running the SPI code in Python

In [313]:
!ncpdq -a time,lat,lon IMERG_concat.nc4 IMERG_concat_ncpdq0.nc4

INFO: NCO stub function for nc_inq_filter_avail() reports an inquiry on filter availability of HDF5 filter ID = 307. Stub function employed because libnetcdf.a does not contain nc_inq_filter_avail(). Please rebuild NCO against netCDF library version 4.9.0 (released ~20220601) or later to support the capability to find and call filters besides DEFLATE, Shuffle, and Fletcher32.
Exiting...
ncpdq: WARNING nc_cdc_lst_bld() reports nco_inq_filter_avail() did not find Bzip2 filter (with HDF5 filter ID = 307) as an HDF5 shared library filter. This is probably fixable because this filter is supported by all default installations of netCDF version 4.9.0 or higher. HINT: If you build netCDF from source, please be sure it was configured with the following options: "--enable-nczarr" and "--with-plugin-dir=${HDF5_PLUGIN_PATH}". The latter is especially important in netCDF 4.9.0. Also, please be sure the library for the missing filter (e.g., libzstd.a, libblosc.a, libbz2.a) is in an automatically sea

8. Check again the header for ```IMERG_concat_ncpdq0.nc4```

In [314]:
!ncdump -h IMERG_concat_ncpdq0.nc4

netcdf IMERG_concat_ncpdq0 {
dimensions:
	lat = 102 ;
	lon = 119 ;
	time = UNLIMITED ; // (266 currently)
	bnds = 2 ;
variables:
	float lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "Latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
	float lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "Longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
	float precipitationCal(time, lat, lon) ;
		precipitationCal:long_name = "Daily accumulated precipitation (combined microwave-IR) estimate" ;
		precipitationCal:units = "mm" ;
		precipitationCal:_FillValue = -9999.9f ;
		precipitationCal:missing_value = -9999.9f ;
		precipitationCal:cell_methods = "time: mean time: sum" ;
	double time(time) ;
		time:standard_name = "time" ;
		time:bounds = "time_bnds" ;
		time:units = "days since 1970-01-01 00:00:00Z" ;
		time:calendar = "standard" ;
		time:axis = "T" ;
		time:cell_methods = "time: mean" ;
	double time_bnds(time, bnds) ;
		time_bnds:cell_metho

9. And the variables for ```precipitation``` is ```time,lat,lon```, it means the result is correct. But the unit still in ```mm/hr```, we need to edit to ```mm```
    
   

10. SPI code does not recognized unit ```mm/day``` or ```mm/month```, we need to edit into ```mm```. To edit the unit attribute names, we will use ```ncatted``` command, follow below code:

In [315]:
!ncatted -a units,precipitationCal,modify,c,'mm' IMERG_concat_ncpdq0.nc4 IMERG_concat_ncpdq1.nc4

11. Check again the header for IMERG_concat_ncpdq1.nc4, to make sure everything is correct.

In [316]:
!ncdump -h IMERG_concat_ncpdq1.nc4

netcdf IMERG_concat_ncpdq1 {
dimensions:
	lat = 102 ;
	lon = 119 ;
	time = UNLIMITED ; // (266 currently)
	bnds = 2 ;
variables:
	float lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "Latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
	float lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "Longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
	float precipitationCal(time, lat, lon) ;
		precipitationCal:long_name = "Daily accumulated precipitation (combined microwave-IR) estimate" ;
		precipitationCal:units = "mm" ;
		precipitationCal:_FillValue = -9999.9f ;
		precipitationCal:missing_value = -9999.9f ;
		precipitationCal:cell_methods = "time: mean time: sum" ;
	double time(time) ;
		time:standard_name = "time" ;
		time:bounds = "time_bnds" ;
		time:units = "days since 1970-01-01 00:00:00Z" ;
		time:calendar = "standard" ;
		time:axis = "T" ;
		time:cell_methods = "time: mean" ;
	double time_bnds(time, bnds) ;
		time_bnds:cell_metho

12. And the ```units``` already in ```mm```
    
  

13. Once this has completed the dataset can be used as input to this package for computing SPI. From above picture, some of the precipitation attribute are still wrong: ```DimensionNames``` and ```Units```. We can leave it as is, SPI code will only read ```units``` and ```variables``` ```precipitation(time,lat,lon)```

## 4. Running SPI code

1. We will use ```IMERG_concat_ncpdq1.nc4``` as ```input```, let's move this data to ```Input``` folder.
    - Move using ```mv oldlocation newlocation``` command: ```mv /IMERG_mm_month/IMERG_concat_ncpdq1.nc4 /Input/IMERG_concat_ncpdq1.nc4```

In [317]:
!mv /content/drive/MyDrive/IWMI/IMERG_DailyData/IMERG_mm_month/IMERG_concat_ncpdq1.nc4 /content/drive/MyDrive/IWMI/IMERG_DailyData/Input/IMERG_concat_ncpdq1.nc4

2. Run the code

In [318]:
# new code
!conda install -c anaconda numpy

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - numpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |         openblas          48 KB  anaconda
    ca-certificates-2022.4.26  |       h06a4308_0         132 KB  anaconda
    certifi-2022.6.15          |   py37h06a4308_0         156 KB  anaconda
    conda-4.13.0               |   py37h06a4308_0         990 KB  anaconda
    numpy-1.21.5               |   py37hf838250_3          10 KB  anaconda
    numpy-base-1.21.5          |   py37h1e6e340_3         6.1 MB  anaconda
    ------------------------------------------------------------
                                       

In [319]:
# new code
!conda upgrade numpy

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - numpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |            2_gnu          23 KB  conda-forge
    brotlipy-0.7.0             |py37h540881e_1004         342 KB  conda-forge
    charset-normalizer-2.1.0   |     pyhd8ed1ab_0          35 KB  conda-forge
    colorama-0.4.5             |     pyhd8ed1ab_0          18 KB  conda-forge
    conda-package-handling-1.8.1|   py37h540881e_1         1.0 MB  conda-forge
    cryptography-37.0.4        |   py37h38fbfac_0         1.5 MB  conda-forge
    idna-3.3                   |     pyhd8ed1ab_0          55 KB  conda-f

In [320]:
!process_climate_indices --index spi --periodicity monthly --netcdf_precip /content/drive/MyDrive/IWMI/IMERG_DailyData/Input/IMERG_concat_ncpdq1.nc4 --var_name_precip precipitationCal --output_file_base /content/drive/MyDrive/IWMI/IMERG_DailyData/Output/IMERG --scales 1 2 3 6 9 12 --calibration_start_year 2000 --calibration_end_year 2022 --multiprocessing all

2022-07-23  06:57:25 INFO Start time:    2022-07-23 06:57:25.011990
2022-07-23  06:57:25 INFO Computing 1-month SPI/Pearson
2022-07-23  06:58:08 INFO Computing 1-month SPI/Gamma
2022-07-23  06:58:30 INFO Computing 2-month SPI/Pearson
2022-07-23  06:59:10 INFO Computing 2-month SPI/Gamma
2022-07-23  06:59:32 INFO Computing 3-month SPI/Pearson
2022-07-23  07:00:11 INFO Computing 3-month SPI/Gamma
2022-07-23  07:00:35 INFO Computing 6-month SPI/Pearson
2022-07-23  07:01:13 INFO Computing 6-month SPI/Gamma
2022-07-23  07:01:35 INFO Computing 9-month SPI/Pearson
2022-07-23  07:02:14 INFO Computing 9-month SPI/Gamma
2022-07-23  07:02:35 INFO Computing 12-month SPI/Pearson
2022-07-23  07:03:12 INFO Computing 12-month SPI/Gamma
2022-07-23  07:03:36 INFO End time:      2022-07-23 07:03:36.390787
2022-07-23  07:03:36 INFO Elapsed time:  0:06:11.378797


In [326]:
# new code
# change the order of the to be readable by QGIS (SPI3, SPI6, SPI9, SPI12)
!ncpdq -a time,lat,lon /content/drive/MyDrive/IWMI/IMERG_DailyData/Output/IMERG_spi_gamma_09.nc /content/drive/MyDrive/IWMI/IMERG_DailyData/Output/QGIS_spi_gamma_09.nc

INFO: NCO stub function for nc_inq_filter_avail() reports an inquiry on filter availability of HDF5 filter ID = 307. Stub function employed because libnetcdf.a does not contain nc_inq_filter_avail(). Please rebuild NCO against netCDF library version 4.9.0 (released ~20220601) or later to support the capability to find and call filters besides DEFLATE, Shuffle, and Fletcher32.
Exiting...
ncpdq: WARNING nc_cdc_lst_bld() reports nco_inq_filter_avail() did not find Bzip2 filter (with HDF5 filter ID = 307) as an HDF5 shared library filter. This is probably fixable because this filter is supported by all default installations of netCDF version 4.9.0 or higher. HINT: If you build netCDF from source, please be sure it was configured with the following options: "--enable-nczarr" and "--with-plugin-dir=${HDF5_PLUGIN_PATH}". The latter is especially important in netCDF 4.9.0. Also, please be sure the library for the missing filter (e.g., libzstd.a, libblosc.a, libbz2.a) is in an automatically sea

## 5. Interpret the result using Panoply

The visualization can be employed with Panoply, which is independent with the google colab.

1. Launch the Panoply desktop application
2. Open the SPI file ```/Output/IMERG_spi_gamma_01.nc``` in Panoply
3. From the Datasets tab select spi_gamma_01 and click Create Plot
4. In the Create Plot window select ‘Create a georeferenced <<Longitude Latitude>> plot’ and click Create
5. When the Plot window opens:
    - Array tab: Change the time into 238 to view the latest/last data ~ Mar 2020
    - Scale tab: Change value on Min -3, Max 3, Major 6, Color Table CB_RdBu_09.cpt
    - Map tab: Change value on Center on Lon 115.0, Lat -8.5, then Zoom in the map through menu-editor Plot > Zoom Plot In few times until Bali island bigger.
    - Overlays tab: Change Overlay 1 to MWDB_Coasts_Countries_1.cnob
    - To explore the values in the array, click on the Array 1 tab at the top of the window

## End